In [28]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import yfinance as yf
import pandas as pd

In [29]:
nifty_symbol = "^NSEI"
banknifty_symbol = "^NSEBANK"

start_date = "2023-02-01"
end_date = "2023-02-10"
nifty_data = yf.download(nifty_symbol, start=start_date, end=end_date)

df = pd.DataFrame(nifty_data)

[*********************100%%**********************]  1 of 1 completed


In [30]:
df["upper_shadow"] = df.apply(lambda row: row['High'] - max(row['Open'], row['Close']),axis=1)
df["lower_shadow"] = df.apply(lambda row: min(row['Open'], row['Close']) - row['Low'],axis=1)
# Calculate body length
df["body"] = abs(df["Close"] - df["Open"])
df["full"] = df['High'] - df['Low']

# Create a new column for the candle type
df["Candle"] = df.apply(lambda row: "Green" if row["Close"] >= row["Open"] else "Red", axis=1)


In [31]:
df['Prev_Open'] = df['Open'].shift(1)
df['Prev_Close'] = df['Close'].shift(1)
df['Prev_High'] = df['High'].shift(1)
df['Prev_Low'] = df['Low'].shift(1)
df['Prev_Candle'] = df['Candle'].shift(1)

df['Next_Open'] = df['Open'].shift(-1)
df['Next_Close'] = df['Close'].shift(-1)
df['Next_High'] = df['High'].shift(-1)
df['Next_Low'] = df['Low'].shift(-1)
df['Next_Candle'] = df['Candle'].shift(-1)


# Engulfing Candle

In [32]:
def engulfing_bearish(row):
    data = row["Prev_Candle"] == "Green" and row["Open"]>=row["Prev_Close"] and row["Close"]<= row["Prev_Open"] and row['Next_Candle']=="Red"
    return data

def engulfing_bullish(row):
    data= row["Prev_Candle"] == "Red" and row["Open"]<=row["Prev_Close"] and row["Close"]>= row["Prev_Open"] and row['Next_Candle']=="Green"
    return data   

In [33]:
df['Engulfing_Bullish'] = df.apply(engulfing_bullish, axis=1)
df['Engulfing_Bearish'] = df.apply(engulfing_bearish, axis=1)

In [34]:
engulfingdf = df[["Open","Close","Candle","Engulfing_Bullish","Engulfing_Bearish"]]

In [35]:
engulfingdf

,Open,Close,Candle,Engulfing_Bullish,Engulfing_Bearish
Date,,,,,
2023-02-01,17811.599609,17616.300781,Red,False,False
2023-02-02,17517.099609,17610.400391,Green,False,False
2023-02-03,17721.750000,17854.050781,Green,False,False
2023-02-06,17818.550781,17764.599609,Red,False,False
2023-02-07,17790.099609,17721.500000,Red,False,False
2023-02-08,17750.300781,17871.699219,Green,False,False
2023-02-09,17885.500000,17893.449219,Green,False,False


# Doji candle

In [36]:
def is_doji(row):
    n = 5 
    data = row["body"] < row['full']/n and row['upper_shadow'] >= n * row["body"] and row['lower_shadow'] >= n * row["body"]
    return data

def is_dragonfly_doji(row):
    n=5
    data =  row["body"] < row['full']/n and row['upper_shadow'] *2 < row['lower_shadow']
    return data

    
def is_gravestone_doji(row):
    n=5
    data =  row["body"] < row['full']/n and row['upper_shadow'] > row['lower_shadow']*2
    return data
# chnage n for body 
# row["body"] < 0.1 * row['full'] = 10% Jetli j body reva joyye 
# chnage number 2 as we need the size of shadow and body 

In [37]:
df['Is_Doji'] = df.apply(is_doji, axis=1)
df['Is_Dragonfly_Doji'] = df.apply(is_dragonfly_doji, axis=1)
df['Is_Gravestone_Doji'] = df.apply(is_gravestone_doji, axis=1)

In [38]:
dojidf = df[["Open","Close","Candle","Is_Doji","Is_Dragonfly_Doji","Is_Gravestone_Doji"]]

In [39]:
dojidf

,Open,Close,Candle,Is_Doji,Is_Dragonfly_Doji,Is_Gravestone_Doji
Date,,,,,,
2023-02-01,17811.599609,17616.300781,Red,False,False,False
2023-02-02,17517.099609,17610.400391,Green,False,False,False
2023-02-03,17721.750000,17854.050781,Green,False,False,False
2023-02-06,17818.550781,17764.599609,Red,False,False,False
2023-02-07,17790.099609,17721.500000,Red,False,False,False
2023-02-08,17750.300781,17871.699219,Green,False,False,False
2023-02-09,17885.500000,17893.449219,Green,False,True,False


# Morning / Evning Star

In [40]:
# Panding Testing
def evning_bearish(row):
    prev_body = abs(row["Prev_Close"] - row["Prev_Open"])
    data = row["Prev_Candle"] == "Green" and row["Next_Candle"] =="Red" and  prev_body > row["body"]
    return data

def morning_bullish(row):
    prev_body = abs(row["Prev_Close"] - row["Prev_Open"])
    data = row["Prev_Candle"] == "Red" and row["Next_Candle"] =="Green" and  prev_body > row["body"]
    return data

In [41]:
df['Evning_Bullish'] = df.apply(evning_bearish, axis=1)
df['Morning_Bearish'] = df.apply(morning_bullish, axis=1)

In [42]:
em_stardf = df[["Open","Close","Candle","Evning_Bullish","Morning_Bearish"]]

In [43]:
em_stardf

,Open,Close,Candle,Evning_Bullish,Morning_Bearish
Date,,,,,
2023-02-01,17811.599609,17616.300781,Red,False,False
2023-02-02,17517.099609,17610.400391,Green,False,True
2023-02-03,17721.750000,17854.050781,Green,False,False
2023-02-06,17818.550781,17764.599609,Red,True,False
2023-02-07,17790.099609,17721.500000,Red,False,False
2023-02-08,17750.300781,17871.699219,Green,False,False
2023-02-09,17885.500000,17893.449219,Green,False,False


# Hammer/Shooting Star

In [44]:
# Need more Optimization

def is_hammer(row):
    data = row['upper_shadow'] *2 < row['lower_shadow'] 
    return data

def is_shooting_star(row):
    data =  row['upper_shadow'] > row['lower_shadow']*2 
    return data



In [45]:
df['Is_Hammer'] = df.apply(is_hammer, axis=1)
df['Is_Shooting_Star'] = df.apply(is_shooting_star, axis=1)

In [46]:
ham_Shoot_df = df[["Open","Close","Candle","Is_Hammer","Is_Shooting_Star"]]

In [47]:
ham_Shoot_df

,Open,Close,Candle,Is_Hammer,Is_Shooting_Star
Date,,,,,
2023-02-01,17811.599609,17616.300781,Red,False,False
2023-02-02,17517.099609,17610.400391,Green,False,False
2023-02-03,17721.750000,17854.050781,Green,True,False
2023-02-06,17818.550781,17764.599609,Red,True,False
2023-02-07,17790.099609,17721.500000,Red,True,False
2023-02-08,17750.300781,17871.699219,Green,False,True
2023-02-09,17885.500000,17893.449219,Green,True,False


# Harami

In [48]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'upper_shadow',
       'lower_shadow', 'body', 'full', 'Candle', 'Prev_Open', 'Prev_Close',
       'Prev_High', 'Prev_Low', 'Prev_Candle', 'Next_Open', 'Next_Close',
       'Next_High', 'Next_Low', 'Next_Candle', 'Engulfing_Bullish',
       'Engulfing_Bearish', 'Is_Doji', 'Is_Dragonfly_Doji',
       'Is_Gravestone_Doji', 'Evning_Bullish', 'Morning_Bearish', 'Is_Hammer',
       'Is_Shooting_Star'],
      dtype='object')

In [49]:
# Panding Testing
def harami_bearish(row):
    prev_body = abs(row["Prev_Close"] - row["Prev_Open"])
    data = row["Prev_Candle"] == "Green" and  prev_body > row["body"]*2 and row["High"]<row["Prev_High"] and row["Close"] < row["Prev_Close"]
    return data

def harami_bullish(row):
    prev_body = abs(row["Prev_Close"] - row["Prev_Open"])
    data = row["Prev_Candle"] == "Red" and  prev_body > row["body"]*2 and row["High"]<row["Prev_High"] and row["Close"] < row["Prev_Close"]
    return data

In [50]:
df['Harami_Bullish'] = df.apply(harami_bearish, axis=1)
df['Harami_Bearish'] = df.apply(harami_bullish, axis=1)

In [51]:
harami_df = df[["Open","Close","Candle","Harami_Bullish","Harami_Bearish"]]

In [57]:
removelist = ['Open',
 'High',
 'Low',
 'Close',
 'Adj Close',
 'Volume',
 'upper_shadow',
 'lower_shadow',
 'body',
 'full',
 'Candle',
 'Prev_Open',
 'Prev_Close',
 'Prev_High',
 'Prev_Low',
 'Prev_Candle',
 'Next_Open',
 'Next_Close',
 'Next_High',
 'Next_Low']

df = df.drop(columns=removelist,axis=1)
last_row = df.iloc[-1]

data = [col for col in last_row.index if last_row[col]]
data

['Is_Dragonfly_Doji', 'Is_Hammer']

In [56]:
df.applay()

AttributeError: 'DataFrame' object has no attribute 'applay'